In [23]:


from typing import Annotated, List
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="qwen-max")

In [24]:
@tool
def multiply_by_max(
        a: Annotated[int, "scale factor"],
        b: Annotated[List[int], "list of ints over which to take maximum"],
) -> int:
    """Multiply a by the maximum of b."""
    return a * max(b)


@tool
def divide_by(a: Annotated[float, "dividend"], b: Annotated[float, "divisor"]) -> float:
    """Divide a by b."""
    return a / b

In [25]:
class Response(BaseModel):
    """Information about function."""
    name: str = Field(None, description="The name of the function")
    args: dict = Field(None, description="The args of the function.")

In [26]:
parser = PydanticOutputParser(pydantic_object=Response)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Information about function.", "properties": {"name": {"default": null, "description": "The name of the function", "title": "Name", "type": "string"}, "args": {"default": null, "description": "The args of the function.", "title": "Args", "type": "object"}}}
```


In [27]:
prompt_string = """
请根据需求从下列函数中选择合适的函数来完成任务，如果没有满足要求的函数，则可以在指定格式的结果中返回空值
函数有：
\n{functions}\n

Wrap the output in `json` tags\n{format_instructions}

问题：
{query}
"""
prompt = ChatPromptTemplate.from_messages(
    [("human", prompt_string)]
).partial(
    functions=[
        multiply_by_max.args_schema.model_json_schema(),
        divide_by.args_schema.model_json_schema()
    ],
    format_instructions=parser.get_format_instructions())

In [28]:
prompt.invoke("请将3乘以一至9的最大值。").to_messages()[0].pretty_print()

================================ Human Message =================================


请根据需求从下列函数中选择合适的函数来完成任务，如果没有满足要求的函数，则可以在指定格式的结果中返回空值
函数有：

[{'description': 'Multiply a by the maximum of b.', 'properties': {'a': {'description': 'scale factor', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'list of ints over which to take maximum', 'items': {'type': 'integer'}, 'title': 'B', 'type': 'array'}}, 'required': ['a', 'b'], 'title': 'multiply_by_max', 'type': 'object'}, {'description': 'Divide a by b.', 'properties': {'a': {'description': 'dividend', 'title': 'A', 'type': 'number'}, 'b': {'description': 'divisor', 'title': 'B', 'type': 'number'}}, 'required': ['a', 'b'], 'title': 'divide_by', 'type': 'object'}]


Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}

In [20]:
chain = prompt | llm
result = chain.invoke("请将3乘以一至9的最大值。")
parser.invoke(result)

Response(name='multiply_by_max', args={'a': 3, 'b': [1, 2, 3, 4, 5, 6, 7, 8, 9]})

In [21]:
chain = prompt | llm
result = chain.invoke("999除一千。")
parser.invoke(result)

Response(name='divide_by', args={'a': 999, 'b': 1000})

In [19]:
chain = prompt | llm
result = chain.invoke("查一下明天中午北京的天气情况")
parser.invoke(result)

Response(name=None, args=None)